# ConvNeXT

ConvNeXT 模型由 Meta FAIR的谢赛宁团队在 [A ConvNet for the 2020s](https://arxiv.org/abs/2201.03545) 一文中提出。ConvNeXT 是一种纯卷积模型（ConvNet），其设计灵感来源于视觉 Transformer，并声称在性能上超越后者。

* [🤗 HuggingFace model card: ConvNeXT](https://huggingface.co/docs/transformers/model_doc/convnext)

论文的摘要：

* **背景**：视觉识别的 ”咆哮的二十年代” 始于视觉 Transformer（ViT）的引入，它迅速超越了卷积神经网络（ConvNets），成为最先进的图像分类模型。然而，标准的 ViT 在应用于通用计算机视觉任务（密集预测），如目标检测和语义分割时，会遇到困难。正是层次化的 Transformer（例如，Swin Transformer）重新引入了多个卷积神经网络先验，使得 Transformer 作为通用视觉骨干变得可行，并在各种视觉任务上展现出卓越的性能。然而，这种混合方法的有效性仍然主要归功于 Transformer 的内在优越性，而不是卷积的固有归纳偏差。

* **主要工作**：在本工作中，我们重新审视了设计空间，并测试了纯卷积神经网络所能达到的极限。我们逐步将标准 ResNet “现代化”，向视觉 Transformer 的设计转变，并发现了几个关键组件，这些组件在性能差异中起到了重要作用。这次探索的结果是一系列被称为 ConvNeXt 的纯卷积神经网络模型。 

* **结果**：完全由标准卷积神经网络模块构建的 ConvNeXts，在准确性和可扩展性方面与 Transformer 相媲美，实现了 87.8%的 ImageNet top-1 准确率，并在 COCO 检测和 ADE20K 分割任务上超越了 Swin Transformer，同时保持了标准卷积神经网络简单高效的特点。

论文中的 **Figure 2** 展示了论文基于 ResNet50 所做的各种改动，包括训练方法与训练超参数方向，模型架构一些宏观方面，模型架构的一些微观方面。

<div style="text-align: center;">
<img src="../images/convNeXT.png" width=500px>
</div>

## 训练调参

研究人员首先发现，虽然架构设计选择至关重要，但训练过程的质量在影响性能结果方面也起着关键作用。受 DeiT 和 Swin Transformers 的启发，ConvNext 紧密适应了它们的训练技术。一些显著的变化包括：

- Epoch：将 Epoch 从原来的 90 个 Epoch 延长到 300 个 Epoch。
- 优化器：使用 AdamW 优化器而不是 Adam 优化器，它们在处理权重衰减的方式上有所不同。
- 正则化：使用随机深度和标签平滑作为正则化技术。
- 数据增强：Mixup（生成随机图像对的加权组合）、Cutmix（切割图像的一部分并用另一幅图像的补丁替换）、RandAugment（应用一系列随机增强，如旋转、平移和剪切）和随机擦除（在图像中随机选择一个矩形区域并用随机值擦除其像素）来增加训练数据。

修改这些训练过程将 ResNet-50 的准确率从 76.1% 提高到 78.8%。

## 框架架构方面的宏观调整

宏观设计指的是在一个系统或模型中做出的高层结构决策和考虑，例如层的排列、计算负载在不同阶段的分布以及整体结构。检查 Swin Transformers 的宏观网络，作者已经确定了两个值得注意的设计考虑因素，这对 ConvNext 的性能有益。

* 阶段计算比例：阶段计算比例指的是神经网络模型各阶段计算负载的分布。ResNet-50 有四个主要阶段，分别有（3, 4, 6, 3）个块，这意味着它的计算比例为 3:4:6:3。为了遵循 Swin Transformer 的 1:1:3:1 的计算比例，研究人员将 ResNet 每个阶段的块数量从（3, 4, 6, 3）调整为（3, 3, 9, 3）。改变阶段计算比例将模型准确率从 78.8% 提高到 79.4%。

* 将 Stem 改为 Patchify：通常，在 ResNet 架构的开始，输入被送入一个步长为 `2` 的 `7×7` 卷积层的 Stem，然后是一个最大池化，用于将图像下采样 `4` 倍。然而，作者发现用具有 `4×4` 核大小和步长为 `4` 的卷积层代替 Stem 更有效，通过非重叠的 `4×4`  Patch 有效地对它们进行卷积。Patchify 起到了将图像下采样 4 倍的相同目的，同时减少了层数。这个 Patchifying 步骤将模型准确率从 79.4% 略微提高到 79.5%。

## ResNeXt 化。

与标准的 ResNet 相比，ResNeXt 在浮点运算次数（FLOPs）和准确率之间展现出了更好的权衡。通过使用 depth-wise 卷积 和 element-wise （1x1）卷积，我们会有 spatial 和 channel mixing 的分离 —— 这也是视觉 Transformers 中发现的一个特征 (SelfAttetion 相当于空间上的信息融合，FFN 相当于通道上的信息融合)。使用 depthwise 卷积会减少 FLOPs 数量，同时降低准确率。然而，通过将通道从 `64` 增加到 `96`，准确率高于原始的 `ResNet-50`，同时保持相似数量的 FLOPs。这种修改将模型准确率从 79.5% 提高到 80.5%。

## Inverted Bottleneck。

在每个 Transformer 块中的一个常见理念是使用 Inverted Bottleneck，其中隐藏层比输入维度大得多。这个理念也已被 MobileNetV2 在计算机视觉中使用和推广。ConvNext 采用了这个理念，输入层有 96 个通道，并将隐藏层增加到 384 个通道。通过使用这种技术，它将模型准确率从 80.5% 提高到 80.6%。

<div style="text-align: center;">
<img src="../images/convnext_inverted_bottleneck.png" width=400px>
</div>

## Large Kernel Size 

视觉 Transformer 性能卓越的一个关键因素是其非局部自注意力，允许更广泛的图像特征感受野。在 Swin Transformers 中，注意力块窗口大小至少设置为 7×7，超过了 ResNext 的 3×3 核尺寸。然而，在调整核尺寸之前，有必要重新定位深度卷积层，如下图所示。这种重新定位使 1×1 层能够有效地处理计算任务，而深度卷积层则作为更非局部的受体发挥作用。这样，网络可以利用合并更大核尺寸卷积的优势。实现 7×7 核尺寸保持准确率在 80.6%，但降低了模型的整体 FLOPs 效率。

<div style="text-align: center;">
<img src="../images/convNext_block.png" width=400px>
</div>

## 模型架构上的微观设计 

除了上述修改之外，作者还对模型进行了一些微观设计的更改。微观设计指的是低级结构决策，例如激活函数的选择和层的细节。一些显著的微观变化包括：

- 激活：用 GELU（高斯误差线性单元）取代 ReLU 激活，并从残差块中除去除两个 1×1 层之间的一个之外的所有 GELU 层。
- 归一化：通过去除两个 BatchNorm 层并将 BatchNorm 替换为 LayerNorm 来减少归一化层，在 conv 1×1 层之前仅保留一个 LayerNorm 层。
- 下采样层：在 ResNet 阶段之间添加一个单独的下采样层。


这些最终的修改将 ConvNext 的准确率从 80.6% 提高到 82.0%。最终的 ConvNext 模型超过了 Swin Transformer 的 81.3% 的准确率。

## 整体架构上的对比


<div style="text-align: center;">
<img src="../images/convnext_detail_arch.png" width=800px>
</div>


# 手动实现 ConvNext 架构

以下实现参考 ConNeXt 的官方 Github 仓库：https://github.com/facebookresearch/ConvNeXt/blob/main/models/convnext.py

## Stem 

ConvNeXt 的 stem 就是一个 pathfy 的过程，所以可以用一个卷积操作来进行计算。

In [14]:
import torch
from torch import nn

x = torch.randn(1, 3, 224, 224)

stem = nn.Conv2d(3, 96, kernel_size=4, stride=4)
hidden_states = stem(x)
print(hidden_states.shape)

torch.Size([1, 96, 56, 56])


## ConvNextBlock

整个 `convNextBlock` 实际是模仿`TransformerLayer`的实现。其中第一个 `7x7` Depthwise 的卷积，就相当于一个局部空间上的自注意力，进行像素与像素之间信息的融合。后面两个 `1x1`的 Pointwise 的卷积本质上就是一个`FFN`。

In [19]:
from timm.models.layers import trunc_normal_, DropPath


class ConvNeXtBlock(nn.Module):
    def __init__(self, dim, drop_path=0, layer_scale_init_value=1e-6):
        super().__init__()
        self.dw_conv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, 4 * dim),
            nn.GELU(),
            nn.Linear(4 * dim, dim),
        )
        self.gamma = (
            nn.Parameter(layer_scale_init_value * torch.ones((dim)), requires_grad=True)
            if layer_scale_init_value > 0
            else None
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

    def forward(self, x):
        short_cut = x
        x = self.dw_conv(x)
        x = torch.permute(x, (0, 2, 3, 1))  # (N,C,H,W) ->(N,H,W,C)
        x = self.norm(x)
        x = self.mlp(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = torch.permute(x, (0, 3, 1, 2))  # (N,H,W,C) ->(N,C,H,W)
        return short_cut + self.drop_path(x)


block = ConvNeXtBlock(dim=96)
hidden_states = block(hidden_states)
print(hidden_states.shape)

torch.Size([1, 96, 56, 56])


## DownSample Layer

在每个 Stage 的开始都会先有一个 downsampling 的操作，将整个 FeatureMap 的 空间尺寸减小，同时把channel数翻倍。它是通过一个卷积核心大小为 2，步长为 2 的卷积来实现的。

```python
downsample_layer = nn.Sequential(
        LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
        nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
)
```

## ConvNextModel

模型实现中的一些细节：

1. drop_prob 的计算。从 0 开始按 stage 线性增加最大到设置的 `drop_path_rate`。
2. 将 stem 也看成一种 downsapling layer，那么整个网络的主干部分就是四个 stage，每个 stage 都是先进行 downsampling，再执行一系列 Block 的计算。
3. 在最后的分类头前面，经过了一个 GlobalAvgPooling 的操作，再经过一个 norm 层。

In [20]:
import torch.nn.functional as F


class LayerNorm(nn.Module):
    r"""LayerNorm that supports two data formats: channels_last (default) or channels_first.
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs
    with shape (batch_size, channels, height, width).
    """

    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.normalized_shape = (normalized_shape,)

    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(
                x, self.normalized_shape, self.weight, self.bias, self.eps
            )
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

In [21]:
class ConvNeXt(nn.Module):

    def __init__(
        self,
        in_chans=3,
        num_classes=1000,
        depths=[3, 3, 9, 3],
        dims=[96, 192, 384, 768],
        drop_path_rate=0.0,
        layer_scale_init_value=1e-6,
        head_init_scale=1.0,
    ):
        super().__init__()

        self.downsample_layers = (
            nn.ModuleList()
        )  # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first"),
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                nn.Conv2d(dims[i], dims[i + 1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = (
            nn.ModuleList()
        )  # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[
                    ConvNeXtBlock(
                        dim=dims[i],
                        drop_path=dp_rates[cur + j],
                        layer_scale_init_value=layer_scale_init_value,
                    )
                    for j in range(depths[i])
                ]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)  # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=0.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(
            x.mean([-2, -1])
        )  # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

In [22]:
model = ConvNeXt()
x = torch.randn(1, 3, 224, 224)
out = model(x)
print(out.shape)

torch.Size([1, 1000])


# Timm 中的 ConvNeXT 实现

In [23]:
import timm
from timm.models.convnext import ConvNeXtBlock, ConvNeXtStage, ConvNeXt

convnext_tiny = timm.create_model("convnext_tiny")
out = convnext_tiny(x)
print(out.shape)

torch.Size([1, 1000])
